In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
os.chdir("..")


In [293]:
import pandas as pd
import mediapipe as mp
from src.data import Video
from src.distances import lcss_dist
from src.models.model import kNNClassifier
from src.resources.joints import JOINTS
from src.visualization import plot_3d_landmarks
from src.models.model import kNNClassifier


In [247]:
DATA_PROCESSED_FILEPATH = "data\\processed"

X_train, y_train = [], []

for (dirpath, _, filenames) in os.walk(DATA_PROCESSED_FILEPATH):
    for frame_name in filenames:
        x = pd.read_csv(os.path.join(dirpath, frame_name), index_col=0)
        x.loc["orientation"] = x.loc["orientation"] / 90
        y = dirpath.split("\\")[-1]
        X_train.append(x)
        y_train.append(y)


In [246]:
X_test = []

for _ in range(6):
    vid = Video(path=1)
    vid.get_landmarks(model_complexity=2, show=True)
    vid.extract_coordinates()
    vid.extract_angles(JOINTS)

    vid._angles.smooth(inplace=True)
    # vid._angles.scale(inplace=True)

    X_test.append(vid._angles.frame)


webcam: 88it [00:11,  7.34it/s]
webcam: 81it [00:13,  5.86it/s]
webcam: 94it [00:12,  7.57it/s]
webcam: 110it [00:14,  7.73it/s]
webcam: 159it [00:17,  9.12it/s]
webcam: 173it [00:21,  7.91it/s]


In [310]:
knn = kNNClassifier(k=3, epsilon=150)
knn.fit(X=X_train, y=y_train)
knn._predict(X_test)

[{'pullup': 0.6666666666666666, 'pushup': 0.3333333333333333},
 {'pullup': 0.6666666666666666, 'pushup': 0.3333333333333333},
 {'squat': 1.0},
 {'squat': 1.0},
 {'pushup': 1.0},
 {'pushup': 0.6666666666666666, 'pullup': 0.3333333333333333}]

In [ ]:
import pandas as pd

predictions = knn._predict(X_test)
pd.DataFrame(predictions)

<img src="https://i.imgur.com/3j8BPdc.png" style="width:800px" >

In [ ]:
vid = Video(path=1)
vid.get_landmarks(show=True, model_complexity=1)
vid.extract_coordinates()

In [ ]:
vid.extract_angles(angle_list=JOINTS["left"])
vid._angles.plot()

In [ ]:
vid._angles.smooth(inplace=True)
vid._angles.plot()

In [ ]:
plot_3d_landmarks(
    landmark_list=vid.landmarks_series[-1],
    connections=mp.solutions.pose.POSE_CONNECTIONS,
)
